In [56]:
# http://naeb.brit.org/uses/ 

import pandas as pd
# import camelot

botany_list = pd.read_html("http://naeb.brit.org/uses/")

# botany_df = pd.DataFrame(botany_list)

botany_df = botany_list[0]

botany_df.columns



Int64Index([0, 1], dtype='int64')

In [55]:
# http://naeb.brit.org/uses/

import pandas as pd
# import camelot

botany_list = pd.read_html("http://naeb.brit.org/uses/species/1")





ValueError: No tables found